
<div style="background-color: lightblue; padding: 60px;">
    <h1><b>Acquire Data through Web Scraping
</b></h1>
</div>


In [34]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import acquire as a

# Exercises

# 2. News Articles
We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.
Write a function that scrapes the news articles for the following topics:
Business Sports Technology Entertainment The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:
{ 'title': 'The article title', 'content': 'The article content', 'category': 'business' # for example } 

Hints:
   - Start by inspecting the website in your browser. Figure out which elements will be useful. 
   - Start by creating a function that handles a single article and produces a dictionary like the one above.      - Next create a function that will find all the articles on a single page and call the function you created in the last step for every article on the page. 
   - Now create a function that will use the previous two functions to scrape the articles from all the pages that you need, and do any additional processing that needs to be done.

In [35]:
# construct a list of urls to go to:
base_url = 'https://inshorts.com/en/read/'
categories = [
    'business',
    'entertainment',
    'technology',
    'sports'
]

In [21]:
[print(requests.get(base_url + cat)) for cat in categories]

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


[None, None, None, None]

In [ ]:
# looks like this is a valid way to traverse the web page.
# lets go to one of the categories and figure out a way to grab the needed content

In [23]:
#tag=div, itemprop="articleBody"

In [36]:
test_soup = BeautifulSoup(requests.get(base_url+categories[0]).text, 'html.parser')


In [25]:
# test out what we found in inspector to see if it matches what we want:
test_soup.find_all('div', itemprop='articleBody')[0].text

'A special PMLA court in Mumbai has allowed Jet Airways Founder Naresh Goyal, an accused in an alleged fraud of ₹538 crore at the Canara Bank, to have home-cooked food inside prison. "Naresh Goyal is permitted to avail home-cooked food on a daily basis, as prayed, at his own risk and the risk of his family/relatives," the court order said.'

In [37]:
test_soup.find_all('span', itemprop='headline')[0].text


"Jet Airways Founder Naresh Goyal allowed home food in jail at 'own risk' "

In [ ]:
# looks like we have a way to grab the headlines and article bodies from the page
# lets see if they match up:


In [38]:
len(test_soup.find_all('div', itemprop='articleBody')) == len(test_soup.find_all('span', itemprop='headline'))


True

In [ ]:
time.time()

In [30]:
 # lets build out a looping structure in order to build out that dataframe!!!
# initialize a DataFrame for containing the contents of our articles
# since we on each page are getting back a list of articles
# and a list of titles,
# it appears a bit easier to initialize a DF
# that we can concatenate since we will be getting 
# multiple rows of content off of each page
# df = pd.DataFrame({'title': titles, 'body': bodies, 'category': cat})
all_articles = pd.DataFrame(columns=['title', 'body','category'])
t_sum = 0
for category in categories:
    t_0 = time.time()
    print(f'Grabbing contents for {category}.')
    # construct a url based on our base concatenated with the cat
    category_url = base_url + category
    # get the response text from the constructed url
    raw_content = requests.get(category_url).text
    #turn the content into soup:
    soup = BeautifulSoup(raw_content, 'html.parser')
    # title content:
    titles = [
        element.text for element in soup.find_all(
            'span', itemprop='headline')
    ]
    # body content:
    bodies = [
        element.text for element in soup.find_all(
            'div', itemprop='articleBody')
    ]
    # category (its already here :) ) 
    category_df = pd.DataFrame(
    {
        'title': titles,
        'body': bodies,
        'category': category
    })
    all_articles = pd.concat(
    [
        all_articles,
        category_df
    ],
    axis=0,
    ignore_index=True)
    t_n = time.time()
    t_delta = t_n - t_0
    print(f'Time to grab contents of {category}: {round(t_delta, 2)} seconds')
    t_sum += t_delta
print('Job finished!')
print(f'It took {round((t_sum / 60), 2)} minutes to execute scraping')

Grabbing contents for business.
Time to grab contents of business: 1.37 seconds
Grabbing contents for entertainment.
Time to grab contents of entertainment: 1.33 seconds
Grabbing contents for technology.
Time to grab contents of technology: 1.5 seconds
Grabbing contents for sports.
Time to grab contents of sports: 1.27 seconds
Job finished!
It took 0.09 minutes to execute scraping


In [43]:

all_articles

,title,body,category
0,Jet Airways Founder Naresh Goyal allowed home ...,A special PMLA court in Mumbai has allowed Jet...,business
1,"Wipro's staff count falls for 4th quarter, att...",Wipro reported a fall in its employee headcoun...,business
2,"Wipro's revenue in Q2 falls to ₹22,516 cr, pro...",IT major Wipro reported a 0.1% fall in its rev...,business
3,"Will march them off to Tihar, they'll know SC'...",Chief Justice of India (CJI) DY Chandrachud re...,business
4,Goldman Sachs CEO David Solomon quits working ...,"David Solomon, the 61-year-old CEO of investme...",business
5,Rajnigandha-parent to buy Noida's GIP Mall for...,"DS Group, known for brands like Rajnigandha an...",business
6,India to respond to concerns on laptop import ...,India will respond to concerns raised by some ...,business
7,Amazon to become rival Microsoft's customer in...,Microsoft is preparing to make its rival Amazo...,business
8,DA hiked by 4% for central govt employees,The Union Cabinet has approved a proposal to r...,business
9,J&J weighs 3rd bankruptcy bid to settle $8.9-b...,Johnson & Johnson is weighing a third attempt ...,business


In [45]:
# Write the above one as a function in acquire.py and call it

# Testing the function:

base_url = 'https://inshorts.com/en/read/'
categories = [
    'business',
    'entertainment',
    'technology',
    'sports'
]

a.scrape_news_articles(categories, base_url)


Grabbing contents for business.
Time to grab contents of business: 0.1 seconds
Grabbing contents for entertainment.
Time to grab contents of entertainment: 0.19 seconds
Grabbing contents for technology.
Time to grab contents of technology: 1.0 seconds
Grabbing contents for sports.
Time to grab contents of sports: 0.17 seconds
Job finished!
It took 0.02 minutes to execute scraping


,title,body,category
0,Jet Airways Founder Naresh Goyal allowed home ...,A special PMLA court in Mumbai has allowed Jet...,business
1,"Wipro's staff count falls for 4th quarter, att...",Wipro reported a fall in its employee headcoun...,business
2,"Wipro's revenue in Q2 falls to ₹22,516 cr, pro...",IT major Wipro reported a 0.1% fall in its rev...,business
3,"Will march them off to Tihar, they'll know SC'...",Chief Justice of India (CJI) DY Chandrachud re...,business
4,Goldman Sachs CEO David Solomon quits working ...,"David Solomon, the 61-year-old CEO of investme...",business
5,Rajnigandha-parent to buy Noida's GIP Mall for...,"DS Group, known for brands like Rajnigandha an...",business
6,India to respond to concerns on laptop import ...,India will respond to concerns raised by some ...,business
7,Amazon to become rival Microsoft's customer in...,Microsoft is preparing to make its rival Amazo...,business
8,DA hiked by 4% for central govt employees,The Union Cabinet has approved a proposal to r...,business
9,J&J weighs 3rd bankruptcy bid to settle $8.9-b...,Johnson & Johnson is weighing a third attempt ...,business


In [ ]:
# or

In [4]:
is_url1 = 'https://inshorts.com/en/read/business'
is_url2 = 'https://inshorts.com/en/read/sports'   
is_url3 = 'https://inshorts.com/en/read/technology'
is_url4 = 'https://inshorts.com/en/read/entertainment'  

In [5]:
user_agents = [
  "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0",
  "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
  "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0"
  ]
random_user_agent = random.choice(user_agents)

# create response object
headers = {'User-Agent': random_user_agent}

is_url1_response = requests.get(is_url1, headers=headers)
is_url2_response = requests.get(is_url2, headers=headers)
is_url3_response = requests.get(is_url3, headers=headers)
is_url4_response = requests.get(is_url4, headers=headers)


In [6]:
print('url1_response',is_url1_response)
print('url2_response',is_url2_response)
print('url3_response',is_url3_response)
print('url4_response',is_url4_response)

url1_response <Response [200]>
url2_response <Response [200]>
url3_response <Response [200]>
url4_response <Response [200]>


In [9]:
# turn the contents into soup:
is_soup_url1 = BeautifulSoup(is_url1_response.text, 'html.parser')
is_soup_url2 = BeautifulSoup(is_url2_response.text, 'html.parser')
is_soup_url3 = BeautifulSoup(is_url3_response.text, 'html.parser')
is_soup_url4 = BeautifulSoup(is_url4_response.text, 'html.parser')

<span itemprop="headline" class="S2DdZEgzkqC9bYeTJUGw">HDFC Bank's Vigil Aunty ad gets criticism for stop-sign bindi, people call it 'anti-Hindu'</span>

In [10]:
# Title of all articles
title_business_news = is_soup_url1.find_all('span', itemprop="headline")
title_sports_news = is_soup_url2.find_all('span', itemprop="headline")
title_technology_news = is_soup_url3.find_all('span', itemprop="headline")
title_entertainment_news = is_soup_url4.find_all('span', itemprop="headline")

<div itemprop="articleBody" class="Hxtmf9GvkV8Ti6V0GUSn">HDFC Bank's latest advertisement featuring Vigil Aunty is receiving criticism online. The advertisement, which aims to raise awareness regarding financial fraud, features Anuradha Menon as Vigil Aunty with a stop-sign like bindi. Some X users called the ad "anti-Hindu" and one of them also wrote, "Is this how you represent Hindu culture with a stop sign on a woman's head?"</div>

In [11]:
# ArticleBody of all articles 
contents_business_news = is_soup_url1.find_all('div', itemprop="articleBody")
contents_sports_news = is_soup_url2.find_all('div', itemprop="articleBody")
contents_technology_news = is_soup_url3.find_all('div', itemprop="articleBody")
contents_entertainment_news = is_soup_url4.find_all('div', itemprop="articleBody")

In [13]:
# Create dictionaries for each article on page
business_news = [{'title': title.text, 'content': content.text, 'category': 'business'} for title, content in zip(title_business_news, contents_business_news)]
sports_news = [{'title': title.text, 'content': content.text, 'category': 'sports'} for title, content in zip(title_sports_news, contents_sports_news)]
technology_news = [{'title': title.text, 'content': content.text, 'category': 'technology'} for title, content in zip(title_technology_news, contents_technology_news)]
entertainment_news = [{'title': title.text, 'content': content.text, 'category': 'entertainment'} for title, content in zip(title_entertainment_news, contents_entertainment_news)]


In [14]:
# combine a list of the dictionaries for all article pages
news = business_news + sports_news + technology_news + entertainment_news

In [15]:
news

[{'title': "HDFC Bank's Vigil Aunty ad gets criticism for stop-sign bindi, people call it 'anti-Hindu'",
  'content': 'HDFC Bank\'s latest advertisement featuring Vigil Aunty is receiving criticism online. The advertisement, which aims to raise awareness regarding financial fraud, features Anuradha Menon as Vigil Aunty with a stop-sign like bindi. Some X users called the ad "anti-Hindu" and one of them also wrote, "Is this how you represent Hindu culture with a stop sign on a woman\'s head?"',
  'category': 'business'},
 {'title': 'IMEC big opportunity for investors to partner with India: PM Modi',
  'content': 'PM Narendra Modi at the Global Maritime India Summit said the India-Middle East-Europe Economic Corridor (IMEC) is a big opportunity for investors to partner with India. This corridor will change the face of regional and global trade, he stated. "Construction of next-generation mega ports...island development...and expansion of multimodal hubs will be taken up under this projec